In [1]:
!pip install pyomo

# Install GLPK solver
!apt-get update -qq
!apt-get install -y glpk-utils

# Optional: Install CBC solver as alternative
print("\nInstalling CBC solver...")
!apt-get install -y coinor-cbc

import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import time

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 35 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsuitesparseconfig5 amd64 1:5.10.1+dfsg-4build1 [10.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libamd2 amd64 1:5.10.1+dfsg-4build1 [21.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcolamd2 amd64 1

In [2]:
def create_tournament_model(n):

    # Validate input
    if n % 2 != 0:
        raise ValueError("Number of teams (n) must be even")

    # Create model
    model = pyo.ConcreteModel()

    # Sets
    model.T = pyo.RangeSet(1, n)  # Teams
    model.W = pyo.RangeSet(1, n-1)  # Weeks
    model.P = pyo.RangeSet(1, n//2)  # Periods

    # Parameters
    games_per_pair = 1
    games_per_team_per_week = 1
    games_per_team_per_period = 2
    fixed_matches = []
    for k in range(1,n,2):
        home = k
        away = k+1
        fixed_matches.append((home, away))

    # Decision Variables
    # x[i,j,w,p] = 1 if team i plays at home against team j in week w in period p
    model.x = pyo.Var(model.T, model.T, model.W, model.P, domain=pyo.Binary)

    # Constraints

    # 1. Every team plays with every other team only once
    def games_per_pair_rule(model, i, j):
        if i != j:
            return sum(model.x[i,j,w,p] + model.x[j,i,w,p]
                      for w in model.W for p in model.P) == games_per_pair
        else:
            return pyo.Constraint.Skip

    model.games_per_pair_constraint = pyo.Constraint(
        model.T, model.T, rule=games_per_pair_rule
    )

    # 2. Every team plays once a week
    def games_per_team_per_week_rule(model, t, w):
        return sum(model.x[t,j,w,p] + model.x[j,t,w,p]
                  for j in model.T for p in model.P) == games_per_team_per_week

    model.games_per_team_per_week_constraint = pyo.Constraint(
        model.T, model.W, rule=games_per_team_per_week_rule
    )

    # 3. Every team plays at most twice in the same period over the tournament
    def games_per_team_per_period_rule(model, t, p):
        return sum(model.x[t,j,w,p] + model.x[j,t,w,p]
                  for j in model.T for w in model.W) <= games_per_team_per_period

    model.games_per_team_per_period_constraint = pyo.Constraint(
        model.T, model.P, rule=games_per_team_per_period_rule
    )

    # 5. In each (week;period) slot a game have to be played
    def one_play_per_period(model, w,p):
        return sum(model.x[i,j,w,p] for i in model.T for j in model.T if i != j) == 1

    model.one_play_per_period = pyo.Constraint(
        model.W, model.P,  rule=one_play_per_period
    )

    # Symmetry breaking constraints to speed up solving
    # fix the first week
    for period, (home_team, away_team) in enumerate(fixed_matches, 1):
        model.x[home_team, away_team, 1, period].fix(1)

    # Objective: Balance the number of home and away games of each team
    model.home_away_diff = pyo.Var(domain=pyo.NonNegativeIntegers)

    def home_away_balance_pos(model,t):
        home_games = sum(model.x[t,j,w,p]
                        for j in model.T for w in model.W for p in model.P)
        away_games = sum(model.x[j,t,w,p]
                        for j in model.T for w in model.W for p in model.P)
        return home_games - away_games <= model.home_away_diff

    def home_away_balance_neg(model,t):
        home_games = sum(model.x[t,j,w,p]
                        for j in model.T for w in model.W for p in model.P)
        away_games = sum(model.x[j,t,w,p]
                        for j in model.T for w in model.W for p in model.P)
        return away_games - home_games <= model.home_away_diff


    model.home_away_pos = pyo.Constraint(model.T, rule=home_away_balance_pos)
    model.home_away_neg = pyo.Constraint(model.T, rule=home_away_balance_neg)

    #Lower bound setted to 1
    def lower_bound_objective(model):
        return model.home_away_diff>=1

    model.lower_buond_objective = pyo.Constraint(rule=lower_bound_objective)

    #Upper bound setted to n-1
    def upper_bound_objective(model):
        return model.home_away_diff<=n-1

    model.upper_buond_objective = pyo.Constraint(rule=upper_bound_objective)

    # Objective function
    model.obj = pyo.Objective(expr=model.home_away_diff, sense=pyo.minimize)

    return model

In [3]:
def print_tournament_schedule(model):

    # First, collect all games
    games = []
    for w in model.W:
        for p in model.P:
            for i in model.T:
                for j in model.T:
                    if i != j and pyo.value(model.x[i,j,w,p]) > 0.5:
                        games.append({
                            'week': w,
                            'period': p,
                            'home': i,
                            'away': j
                        })

    print("\n" + "="*80)
    print(" " * 25 + "🏆 TOURNAMENT SCHEDULE 🏆")
    print("="*80)

    # Main Schedule Table: Periods (rows) × Weeks (columns)
    print("\n📊 SCHEDULE TABLE (Periods × Weeks)")
    print("-" * 80)

    # Calculate column width based on number of weeks
    weeks = list(model.W)
    periods = list(model.P)
    col_width = max(15, 60 // len(weeks))  # Adjust column width based on number of weeks

    # Create and print header
    header = "Period".ljust(7) + "│"
    for w in weeks:
        header += f" Week {w}".center(col_width) + "│"
    print("┌" + "─" * 7 + "┬" + ("─" * col_width + "┬") * len(weeks))
    print("│" + header[:-1])
    print("├" + "─" * 7 + "┼" + ("─" * col_width + "┼") * len(weeks))

    # Print each period row
    for p in periods:
        row = f"   {p}   │"

        for w in weeks:
            # Find games for this period and week
            period_week_games = [g for g in games if g['period'] == p and g['week'] == w]

            if period_week_games:
                # Format games for this cell
                games_in_cell = []
                for game in period_week_games:
                    games_in_cell.append(f"{game['home']}v{game['away']}")

                cell_content = ", ".join(games_in_cell)

                # Truncate if too long for column
                if len(cell_content) > col_width - 2:
                    cell_content = cell_content[:col_width-5] + "..."

                row += f" {cell_content}".ljust(col_width) + "│"
            else:
                row += " ".ljust(col_width) + "│"

        print("│" + row[:-1])

    # Bottom border
    print("└" + "─" * 7 + "┴" + ("─" * col_width + "┴") * len(weeks))


    # Home/Away balance per team
    print(f"\n⚖️  HOME/AWAY BALANCE")
    print("-" * 30)
    print("Team | Home | Away | Diff")
    print("-----|------|------|------")

    for t in model.T:
        home_count = len([g for g in games if g['home'] == t])
        away_count = len([g for g in games if g['away'] == t])
        diff_count = abs(home_count - away_count)
        print(f"  {t}  |  {home_count}   |  {away_count}   |   {diff_count}")

    print("="*80)

In [4]:
def solve_tournament(n, solver_name='cbc', time_limit=300, tee=False):
    """
    Solve the tournament scheduling problem

    Parameters:
    n: number of teams (must be even)
    solver_name: name of the solver to use
    time_limit: maximum time in seconds (default: 300)
    """

    # Create model
    model = create_tournament_model(n)

    # Solver
    solver = SolverFactory(solver_name)

    # Set time limit based on solver type
    if solver_name.lower() in ['gurobi', 'grb']:
        solver.options['TimeLimit'] = time_limit
    elif solver_name.lower() in ['cplex', 'cplexdirect']:
        solver.options['timelimit'] = time_limit
    elif solver_name.lower() in ['glpk']:
        solver.options['tmlim'] = time_limit
    elif solver_name.lower() in ['cbc']:
        solver.options['sec'] = time_limit
    elif solver_name.lower() in ['scip']:
        solver.options['limits/time'] = time_limit
    else:
        print(f"Warning: Time limit setting not configured for solver {solver_name}")

    # Record start time
    start_time = time.time()

    # Solve with time tracking
    print(f"Starting optimization with {time_limit}s time limit...")
    results = solver.solve(model, tee=tee)

    # Calculate elapsed time
    elapsed_time = time.time() - start_time
    print(f"Solver finished in {elapsed_time} seconds")

    # Print results
    if results.solver.termination_condition == pyo.TerminationCondition.optimal:
        print("\nOptimal solution found!")
        print(f"Objective value: {pyo.value(model.obj)}")
        print_tournament_schedule(model)

    else:
        print(f"Solver terminated with condition: {results.solver.termination_condition}")

    return model, results

In [5]:
try:
    model, results = solve_tournament(10, solver_name='cbc', time_limit=300, tee=False)
except Exception as e:
    print(f"Error: {e}")

Starting optimization with 300s time limit...
Solver finished in 14.84567904472351 seconds

Optimal solution found!
Objective value: 1.0

                         🏆 TOURNAMENT SCHEDULE 🏆

📊 SCHEDULE TABLE (Periods × Weeks)
--------------------------------------------------------------------------------
┌───────┬───────────────┬───────────────┬───────────────┬───────────────┬───────────────┬───────────────┬───────────────┬───────────────┬───────────────┬
│Period │     Week 1    │     Week 2    │     Week 3    │     Week 4    │     Week 5    │     Week 6    │     Week 7    │     Week 8    │     Week 9    
├───────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼───────────────┼
│   1   │ 1v2           │ 7v3           │ 10v7          │ 8v4           │ 6v9           │ 3v9           │ 4v6           │ 1v10          │ 2v5           
│   2   │ 3v4           │ 6v2           │ 6v8           │ 7v9           │ 10v3    

| Basic |      |   |   |   |
|-------|------|---|---|---|
| 6     | 0.9s |   |   |   |
| 8     | ---- |   |   |   |
|       |      |   |   |   |

-

| Lower bound added |         |   |   |   |
|-------------------|---------|---|---|---|
| Teams             | Time(s) |   |   |   |
| 6                 | 0.28    |   |   |   |
| 8                 | 2.18    |   |   |   |
| 10                | 77.26   |   |   |   |
| 12                | ----    |   |   |   |

-

| Fixing first week |         |   |   |   |
|-------------------|---------|---|---|---|
| Teams             | Time(s) |   |   |   |
| 6                 | 0.3     |   |   |   |
| 8                 | 3.9     |   |   |   |
| 10                | 8.78    |   |   |   |
| 12                | ----    |   |   |   |